In [1]:
!pip install opensoundscape==0.6.2
from opensoundscape.preprocess.preprocessors import BasePreprocessor
import torch
import pandas as pd
from pathlib import Path
import numpy as np
import pandas as pd
from opensoundscape.torch.models import cnn
from opensoundscape.torch.architectures import cnn_architectures

In [33]:
#COSMOS_BirdNET-Lite_Labels_100.csv
name = "autoamted_cosmos_tweety_to_file"
main_name ="./" +name + ".csv"#"./COSMOS_Microfaune-Filename_Labels_100.csv"#"./COSMOS_BirdNET-Lite-Filename_Labels_05Conf.csv" #'./COSMOS_BirdNET-Lite_Labels_05Conf.csv' #Tweety birdnet, #tweetyfilename
df = pd.read_csv(main_name, sep=',')
test_name = "./" +name + "_TESTING.csv" #"./COSMOS_BirdNET-Lite_Labels_100_TESTING.csv"
train_name ="./" +name + "_TRAINING.csv" #"./COSMOS_BirdNET-Lite_Labels_100_TRAINING.csv"
THIS_IS_NEW_DATA = False
#df['IN FILE'] = [f.split('/')[-1] for f in df['FILENAME']]
print(df["IN FILE"])

0        XC100027 - Southern Nightingale-Wren - Microce...
1        XC100027 - Southern Nightingale-Wren - Microce...
2        XC100027 - Southern Nightingale-Wren - Microce...
3        XC100027 - Southern Nightingale-Wren - Microce...
4        XC100027 - Southern Nightingale-Wren - Microce...
                               ...                        
55642    XC9881 - Blue-grey Tanager - Thraupis episcopu...
55643    XC9881 - Blue-grey Tanager - Thraupis episcopu...
55644    XC9881 - Blue-grey Tanager - Thraupis episcopu...
55645    XC9881 - Blue-grey Tanager - Thraupis episcopu...
55646    XC99284 - Slate-throated Whitestart - Myioboru...
Name: IN FILE, Length: 55647, dtype: object


/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [34]:
ef = pd.read_csv('./cosmos_labeled_data_files_added.csv')
ef['IN FILE'] = ef['IN FILE'].apply(lambda x : x.replace("_", " "))
gtClips = np.array(ef['IN FILE'].unique())
ef.shape

(5192, 17)

In [35]:
ef["IN FILE"][0]

'XC87449 - Scaly-breasted Woodpecker - Celeus grammicus subcervinus.mp3'

In [36]:

j = 0
ff = ef#pd.DataFrame(columns=df.columns)
#for i, row in ef.iterrows():
#    if(row['IN FILE'] in gtClips):
#        # print(row['IN FILE'])
##        ff.loc[j] = row
#        df.drop(i, inplace=True)
#        j += 1
#print(df.shape)
try:
    df = df[df['CONFIDENCE'] >= 0.8]
except:
    pass

In [37]:
ff

,Unnamed: 0,IN FILE,CLIP LENGTH,OFFSET,DURATION,MAX FREQ,MIN FREQ,SAMPLE RATE,MANUAL ID,TIME_SPENT,LAST MOD BY,CONFIDENCE,RETIRED,COUNTED,FOLDER,COMMON,SCIENTIFIC
0,0,XC87449 - Scaly-breasted Woodpecker - Celeus g...,23.2956,0.7025,0.9199,24000,0,44100,Celeus grammicus,365.822,lynette 07/27/2022-17:52:13,0,False,0,./COSMOS,Scaly-breasted Woodpecker,Celeus grammicus
1,1,XC87449 - Scaly-breasted Woodpecker - Celeus g...,23.2956,4.6223,0.9400,24000,0,44100,Celeus grammicus,365.822,lynette 07/27/2022-17:52:13,0,False,0,./COSMOS,Scaly-breasted Woodpecker,Celeus grammicus
2,2,XC87449 - Scaly-breasted Woodpecker - Celeus g...,23.2956,11.4420,0.9300,24000,0,44100,Celeus grammicus,176.450,lynette 07/27/2022-17:52:00,0,False,0,./COSMOS,Scaly-breasted Woodpecker,Celeus grammicus
3,3,XC87449 - Scaly-breasted Woodpecker - Celeus g...,23.2956,18.4117,0.8700,24000,0,44100,Celeus grammicus,365.822,lynette 07/27/2022-17:52:13,0,False,0,./COSMOS,Scaly-breasted Woodpecker,Celeus grammicus
4,4,XC87449 - Scaly-breasted Woodpecker - Celeus g...,23.2956,21.9915,0.7600,24000,0,44100,Celeus grammicus,176.450,lynette 07/27/2022-17:52:00,0,False,0,./COSMOS,Scaly-breasted Woodpecker,Celeus grammicus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5187,648,XC27942 - Yellow-olive Flatbill - Tolmomyias s...,31.3404,15.4774,0.4799,24000,0,44100,Tolmomyias sulphurescens,76.420,Taejun 07/27/2022-18:16:18,0,False,0,./COSMOS,Yellow-olive Flatbill,Tolmomyias sulphurescens
5188,649,XC27942 - Yellow-olive Flatbill - Tolmomyias s...,31.3404,5.4311,0.5899,24000,0,44100,Tolmomyias sulphurescens,76.420,Taejun 07/27/2022-18:16:18,0,False,0,./COSMOS,Yellow-olive Flatbill,Tolmomyias sulphurescens
5189,650,XC27942 - Yellow-olive Flatbill - Tolmomyias s...,31.3404,7.1648,0.6299,24000,0,44100,Tolmomyias sulphurescens,76.420,Taejun 07/27/2022-18:16:18,0,False,0,./COSMOS,Yellow-olive Flatbill,Tolmomyias sulphurescens
5190,651,XC27942 - Yellow-olive Flatbill - Tolmomyias s...,31.3404,10.3063,0.4499,24000,0,44100,Tolmomyias sulphurescens,76.420,Taejun 07/27/2022-18:16:18,0,False,0,./COSMOS,Yellow-olive Flatbill,Tolmomyias sulphurescens


In [38]:
len(df['IN FILE'].unique())

/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2710

In [39]:
df['IN FILE'] = ['./Cosmos_data/Training/' + f for f in df['IN FILE']]
ff['IN FILE CURRENT'] = ['./Cosmos_data/Training_Xeno_Canto_2022/' + f for f in ff['IN FILE']]
ff['IN FILE'] = ['./Cosmos_data/Testing_Xeno_Canto_2022/' + f for f in ff['IN FILE']]


In [40]:
ef = ff.groupby('IN FILE', group_keys=False).apply(lambda ff: ff.sample(1))
#ef['IN FILE'] = ['./XenoCanto_Data/Testing_Xento_Canto_2022/' + f for f in ef['IN FILE']]
ef

,Unnamed: 0,IN FILE,CLIP LENGTH,OFFSET,DURATION,MAX FREQ,MIN FREQ,SAMPLE RATE,MANUAL ID,TIME_SPENT,LAST MOD BY,CONFIDENCE,RETIRED,COUNTED,FOLDER,COMMON,SCIENTIFIC,IN FILE CURRENT
1980,118,./Cosmos_data/Testing_Xeno_Canto_2022/XC100864...,9.39575,0.0116,2.3307,24000,0,44100,Zonotrichia capensis,28.134,Sean-SHP 07/27/2022-21:21:26,0,False,0,./COSMOS,Rufous-collared Sparrow,Zonotrichia capensis,./Cosmos_data/Training_Xeno_Canto_2022/XC10086...
1858,498,./Cosmos_data/Testing_Xeno_Canto_2022/XC101469...,137.41000,108.5410,0.2600,24000,0,44100,Turdus leucomelas,323.179,jasmine-l 07/27/2022-18:08:25,0,False,0,./COSMOS,Pale-breasted Thrush,Turdus leucomelas,./Cosmos_data/Training_Xeno_Canto_2022/XC10146...
1988,126,./Cosmos_data/Testing_Xeno_Canto_2022/XC101564...,63.97150,37.9235,2.0500,24000,0,44100,Zonotrichia capensis,106.000,korlapatia 07/27/2022-18:17:01,0,False,0,./COSMOS,Rufous-collared Sparrow,Zonotrichia capensis,./Cosmos_data/Training_Xeno_Canto_2022/XC10156...
2536,26,./Cosmos_data/Testing_Xeno_Canto_2022/XC101694...,23.05110,19.6288,2.3494,24000,0,44100,Myioborus miniatus,59.145,Shuchi-D 07/27/2022-18:04:59,0,False,0,./COSMOS,Slate-throated Whitestart,Myioborus miniatus,./Cosmos_data/Training_Xeno_Canto_2022/XC10169...
1361,1,./Cosmos_data/Testing_Xeno_Canto_2022/XC101816...,34.43860,15.2995,0.6399,24000,0,44100,bird,106.274,jasmine-l 07/27/2022-18:01:37,0,False,0,./COSMOS,bird,bird,./Cosmos_data/Training_Xeno_Canto_2022/XC10181...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4651,112,./Cosmos_data/Testing_Xeno_Canto_2022/XC96443 ...,32.29380,29.8367,0.7901,24000,0,44100,bird,100.898,Taejun 07/27/2022-18:02:10,0,False,0,./COSMOS,bird,bird,./Cosmos_data/Training_Xeno_Canto_2022/XC96443...
3150,13,./Cosmos_data/Testing_Xeno_Canto_2022/XC96647 ...,108.87900,51.7025,1.4300,24000,0,48000,Microcerculus marginatus,242.494,cheryl 07/27/2022-18:26:30,0,False,0,./COSMOS,Southern Nightingale-Wren,Microcerculus marginatus,./Cosmos_data/Training_Xeno_Canto_2022/XC96647...
603,11,./Cosmos_data/Testing_Xeno_Canto_2022/XC97532 ...,20.33650,0.0000,20.2971,24000,0,44100,No class of interest,50.523,Aarushi-AK 07/27/2022-18:26:21,0,False,0,./COSMOS,No class of interest,No class of interest,./Cosmos_data/Training_Xeno_Canto_2022/XC97532...
3067,557,./Cosmos_data/Testing_Xeno_Canto_2022/XC97542 ...,4.40037,1.8790,1.2111,24000,0,44100,Myioborus miniatus,24.797,spres 07/28/2022-20:03:54,0,False,0,./COSMOS,Slate-throated Whitestart,Myioborus miniatus,./Cosmos_data/Training_Xeno_Canto_2022/XC97542...


In [41]:
#ef["IN FILE"][2]

/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [42]:
import shutil

def Move_Files(filename):
    file = filename.split("/")
    og_file_path = "/home/shperry/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/" + file[3]
    new_file_path = "/home/shperry/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/" + "/".join(file[2:])
    if(THIS_IS_NEW_DATA):
        shutil.copyfile(og_file_path, new_file_path)
    return new_file_path
#Move_Files('./Cosmos_data/Testing_Xeno_Canto_2022/XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis.mp3')
#shutil.copyfile('Cosmos_data/XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis.mp3', './Cosmos_data/Testing_Xeno_Canto_2022/XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis.mp3')
#shutil.copyfile("/home/shperry/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis.mp3", '/home/shperry/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Testing_Xeno_Canto_2022/XC100864 - Rufous-collared Sparrow - Zonotrichia capensis costaricensis.mp3')
ef["IN FILE"] = ef["IN FILE"].apply(Move_Files)


In [43]:
ef

,Unnamed: 0,IN FILE,CLIP LENGTH,OFFSET,DURATION,MAX FREQ,MIN FREQ,SAMPLE RATE,MANUAL ID,TIME_SPENT,LAST MOD BY,CONFIDENCE,RETIRED,COUNTED,FOLDER,COMMON,SCIENTIFIC,IN FILE CURRENT
1980,118,/home/shperry/passive-acoustic-biodiversity/Op...,9.39575,0.0116,2.3307,24000,0,44100,Zonotrichia capensis,28.134,Sean-SHP 07/27/2022-21:21:26,0,False,0,./COSMOS,Rufous-collared Sparrow,Zonotrichia capensis,./Cosmos_data/Training_Xeno_Canto_2022/XC10086...
1858,498,/home/shperry/passive-acoustic-biodiversity/Op...,137.41000,108.5410,0.2600,24000,0,44100,Turdus leucomelas,323.179,jasmine-l 07/27/2022-18:08:25,0,False,0,./COSMOS,Pale-breasted Thrush,Turdus leucomelas,./Cosmos_data/Training_Xeno_Canto_2022/XC10146...
1988,126,/home/shperry/passive-acoustic-biodiversity/Op...,63.97150,37.9235,2.0500,24000,0,44100,Zonotrichia capensis,106.000,korlapatia 07/27/2022-18:17:01,0,False,0,./COSMOS,Rufous-collared Sparrow,Zonotrichia capensis,./Cosmos_data/Training_Xeno_Canto_2022/XC10156...
2536,26,/home/shperry/passive-acoustic-biodiversity/Op...,23.05110,19.6288,2.3494,24000,0,44100,Myioborus miniatus,59.145,Shuchi-D 07/27/2022-18:04:59,0,False,0,./COSMOS,Slate-throated Whitestart,Myioborus miniatus,./Cosmos_data/Training_Xeno_Canto_2022/XC10169...
1361,1,/home/shperry/passive-acoustic-biodiversity/Op...,34.43860,15.2995,0.6399,24000,0,44100,bird,106.274,jasmine-l 07/27/2022-18:01:37,0,False,0,./COSMOS,bird,bird,./Cosmos_data/Training_Xeno_Canto_2022/XC10181...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4651,112,/home/shperry/passive-acoustic-biodiversity/Op...,32.29380,29.8367,0.7901,24000,0,44100,bird,100.898,Taejun 07/27/2022-18:02:10,0,False,0,./COSMOS,bird,bird,./Cosmos_data/Training_Xeno_Canto_2022/XC96443...
3150,13,/home/shperry/passive-acoustic-biodiversity/Op...,108.87900,51.7025,1.4300,24000,0,48000,Microcerculus marginatus,242.494,cheryl 07/27/2022-18:26:30,0,False,0,./COSMOS,Southern Nightingale-Wren,Microcerculus marginatus,./Cosmos_data/Training_Xeno_Canto_2022/XC96647...
603,11,/home/shperry/passive-acoustic-biodiversity/Op...,20.33650,0.0000,20.2971,24000,0,44100,No class of interest,50.523,Aarushi-AK 07/27/2022-18:26:21,0,False,0,./COSMOS,No class of interest,No class of interest,./Cosmos_data/Training_Xeno_Canto_2022/XC97532...
3067,557,/home/shperry/passive-acoustic-biodiversity/Op...,4.40037,1.8790,1.2111,24000,0,44100,Myioborus miniatus,24.797,spres 07/28/2022-20:03:54,0,False,0,./COSMOS,Slate-throated Whitestart,Myioborus miniatus,./Cosmos_data/Training_Xeno_Canto_2022/XC97542...


In [44]:
#file_name = ef['IN FILE'][2].split('/')
#file_name

/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
#file_name[-1]

In [46]:
from opensoundscape.audio import Audio
import os
clip_duration = 3
clip_overlap = 0
final_clip = None
clip_dir = './temp_clips/Cosmos_Data/Testing_Xeno_Canto_2022/'
# classes = labels
min_label_overlap = 0.1

ef_split_save = pd.DataFrame(columns=['file', 'start_time', 'end_time', 'SAMPLING RATE', 'MANUAL ID'])

cnt = 0
ef = ef.reset_index()
print(ef)
for i, row in ef.iterrows():
    try:
        audio = Audio.from_file(row['IN FILE'])
    except Exception:
        print(row['IN FILE'] + ' not found')
        continue
    file_name = row['IN FILE'].split('/')
    # print(file_name)
    clip_df = audio.split_and_save(
        clip_dir,
        prefix=file_name[-1],
        clip_duration=clip_duration,
        clip_overlap=clip_overlap,
        final_clip='remainder',
        dry_run=(os.path.exists(clip_dir + file_name[-1]))
    )
    clip_df['SAMPLING RATE'] = [44100] * clip_df.shape[0]
    clip_df['MANUAL ID'] = [row['MANUAL ID']] * clip_df.shape[0]
    clip_df.reset_index(inplace=True)
    ef_split_save = ef_split_save.append(clip_df, ignore_index=True)
    print(i, "out of", ef.shape[0])

print(ef_split_save)
ef_split_save.to_csv(test_name)

     index  Unnamed: 0                                            IN FILE  CLIP LENGTH    OFFSET  DURATION  MAX FREQ  MIN FREQ  SAMPLE RATE                 MANUAL ID  TIME_SPENT                     LAST MOD BY  CONFIDENCE  RETIRED  COUNTED    FOLDER                      COMMON                SCIENTIFIC                                    IN FILE CURRENT
0     1980         118  /home/shperry/passive-acoustic-biodiversity/Op...      9.39575    0.0116    2.3307     24000         0        44100      Zonotrichia capensis      28.134    Sean-SHP 07/27/2022-21:21:26           0    False        0  ./COSMOS     Rufous-collared Sparrow      Zonotrichia capensis  ./Cosmos_data/Training_Xeno_Canto_2022/XC10086...
1     1858         498  /home/shperry/passive-acoustic-biodiversity/Op...    137.41000  108.5410    0.2600     24000         0        44100         Turdus leucomelas     323.179   jasmine-l 07/27/2022-18:08:25           0    False        0  ./COSMOS        Pale-breasted Thrush         Turd

/home/shperry/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Testing_Xeno_Canto_2022/XC288633 - Buff-throated Woodcreeper - Xiphorhynchus guttatus eytoni ssp assumed from location.mp3 not found
/home/shperry/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Testing_Xeno_Canto_2022/XC288633 - Buff-throated Woodcreeper - Xiphorhynchus guttatus eytoni.mp3 not found
229 out of 498
230 out of 498
231 out of 498
232 out of 498
233 out of 498
234 out of 498
235 out of 498
236 out of 498
237 out of 498
238 out of 498
239 out of 498
240 out of 498
241 out of 498
/home/shperry/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Testing_Xeno_Canto_2022/XC305391 - Buff-throated Woodcreeper - Xiphorhynchus guttatus guttatoides.mp3 not found
243 out of 498
244 out of 498
245 out of 498
246 out of 498
247 out of 498
248 out of 498
249 out of 498
250 out of 498
251 out of 498
252 out of 498
253 out of 498
254 out o

In [47]:
ef_split_save

/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,file,start_time,end_time,SAMPLING RATE,MANUAL ID
0,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,0.0,3.000000,44100,Zonotrichia capensis
1,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,3.0,6.000000,44100,Zonotrichia capensis
2,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,6.0,9.000000,44100,Zonotrichia capensis
3,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,9.0,9.299592,44100,Zonotrichia capensis
4,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,0.0,3.000000,44100,Turdus leucomelas
...,...,...,...,...,...
6928,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,3.0,4.362449,44100,Myioborus miniatus
6929,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,0.0,3.000000,44100,Zonotrichia capensis
6930,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,3.0,6.000000,44100,Zonotrichia capensis
6931,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,6.0,9.000000,44100,Zonotrichia capensis


In [48]:
import shutil

def Move_Files(filename):
    file = filename.split("/")
    og_file_path = "/home/shperry/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/" + file[3]
    new_file_path = "/home/shperry/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/" + "/".join(file[2:])
    if(not os.path.exists(new_file_path)):
        shutil.copyfile(og_file_path, new_file_path)
    return new_file_path
df = pd.read_csv(main_name, sep=',')
df['IN FILE'] = [f.split('/')[-1] for f in df['IN FILE']]
df['IN FILE'] = ['./Cosmos_Data/Training_Xeno_Canto_2022/' + f for f in df['IN FILE']]
df["IN FILE"] = df["IN FILE"].apply(Move_Files)
df

/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,OFFSET,DURATION,FOLDER,IN FILE,CHANNEL,CLIP LENGTH,SAMPLE RATE,MANUAL ID,CONFIDENCE,FILEPATH
0,0,1.883721,0.116279,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,81.528163,44100,Microcerculus marginatus,0.543653,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
1,1,3.976744,0.046512,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,81.528163,44100,Microcerculus marginatus,0.230806,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
2,2,5.976744,0.046512,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,81.528163,44100,Microcerculus marginatus,0.204163,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
3,3,6.023256,0.046512,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,81.528163,44100,Microcerculus marginatus,0.264257,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
4,4,6.069767,0.116279,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,81.528163,44100,Microcerculus marginatus,0.544028,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
...,...,...,...,...,...,...,...,...,...,...,...
55642,55642,1.511628,0.325581,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,14.700000,44100,Thraupis episcopus,0.943661,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...
55643,55643,1.860465,0.139535,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,14.700000,44100,Thraupis episcopus,0.855924,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...
55644,55644,5.976744,0.372093,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,14.700000,44100,Thraupis episcopus,0.775321,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...
55645,55645,6.627907,0.255814,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,14.700000,44100,Thraupis episcopus,0.736337,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...


In [49]:
df

/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,OFFSET,DURATION,FOLDER,IN FILE,CHANNEL,CLIP LENGTH,SAMPLE RATE,MANUAL ID,CONFIDENCE,FILEPATH
0,0,1.883721,0.116279,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,81.528163,44100,Microcerculus marginatus,0.543653,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
1,1,3.976744,0.046512,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,81.528163,44100,Microcerculus marginatus,0.230806,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
2,2,5.976744,0.046512,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,81.528163,44100,Microcerculus marginatus,0.204163,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
3,3,6.023256,0.046512,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,81.528163,44100,Microcerculus marginatus,0.264257,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
4,4,6.069767,0.116279,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,81.528163,44100,Microcerculus marginatus,0.544028,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
...,...,...,...,...,...,...,...,...,...,...,...
55642,55642,1.511628,0.325581,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,14.700000,44100,Thraupis episcopus,0.943661,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...
55643,55643,1.860465,0.139535,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,14.700000,44100,Thraupis episcopus,0.855924,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...
55644,55644,5.976744,0.372093,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,14.700000,44100,Thraupis episcopus,0.775321,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...
55645,55645,6.627907,0.255814,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,14.700000,44100,Thraupis episcopus,0.736337,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...


In [50]:
def fix_folder_path(filepath):
    folder = "/home/shperry/passive-acoustic-biodiversity/OpenSoundScape/MultiClass Classifier/Cosmos_Data/Training_Xeno_Canto_2022/"
    filename = filepath.split("/")[-1]
    return folder + filename
df["IN FILE"] = df["IN FILE"].apply(fix_folder_path)
df

/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,OFFSET,DURATION,FOLDER,IN FILE,CHANNEL,CLIP LENGTH,SAMPLE RATE,MANUAL ID,CONFIDENCE,FILEPATH
0,0,1.883721,0.116279,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,81.528163,44100,Microcerculus marginatus,0.543653,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
1,1,3.976744,0.046512,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,81.528163,44100,Microcerculus marginatus,0.230806,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
2,2,5.976744,0.046512,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,81.528163,44100,Microcerculus marginatus,0.204163,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
3,3,6.023256,0.046512,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,81.528163,44100,Microcerculus marginatus,0.264257,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
4,4,6.069767,0.116279,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,81.528163,44100,Microcerculus marginatus,0.544028,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC1000...
...,...,...,...,...,...,...,...,...,...,...,...
55642,55642,1.511628,0.325581,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,14.700000,44100,Thraupis episcopus,0.943661,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...
55643,55643,1.860465,0.139535,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,14.700000,44100,Thraupis episcopus,0.855924,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...
55644,55644,5.976744,0.372093,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,14.700000,44100,Thraupis episcopus,0.775321,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...
55645,55645,6.627907,0.255814,C:/Users/Siloux/Desktop/E4E/Cosmos_data/,/home/shperry/passive-acoustic-biodiversity/Op...,0,14.700000,44100,Thraupis episcopus,0.736337,C:/Users/Siloux/Desktop/E4E/Cosmos_data/XC9881...


In [51]:
if (False):
    from opensoundscape.audio import Audio
    clip_duration = 3
    clip_overlap = 0
    final_clip = None
    clip_dir = './temp_clips/Cosmos_Data/Training_Xeno_Canto_2022'
    # classes = labels
    min_label_overlap = 0.1

    df_split_save = pd.DataFrame(columns=['file', 'start_time', 'end_time', 'SAMPLING RATE', 'MANUAL ID', "CONFIDENCE"])

    cnt = 0
    df = df.reset_index()
    for i, row in df.iterrows():
        try:
            audio = Audio.from_file(row['IN FILE'])
        except Exception:
            print(row['IN FILE'] + ' not found')
            continue
        file_name = row['IN FILE'].split('/')
        # print(file_name)
        clip_df = pd.DataFrame()
        offset = row["OFFSET"]
        duration = row["DURATION"]
        new_file = clip_dir + file_name[-1] + str(offset) + "s_" + str(offset+duration)+"s.wav"
        if (not os.path.exists(new_file)):
            clip_df = audio.split_and_save(
                clip_dir,
                prefix=file_name[-1],
                clip_duration=clip_duration,
                clip_overlap=clip_overlap,
                final_clip='remainder',
                dry_run=(os.path.exists(clip_dir + file_name[-1]))
            )
        else:
            tmp_df = pd.DataFrame()
            offset = row["OFFSET"]
            duration = row["DURATION"]
            tmp_df['file'] = new_file
            tmp_df['start_time'] = offset
            tmp_df['end_time'] = offset + duration
            if (clip_df.empty):
                clip_df = tmp_df
            else:
                clip_df = clip_df.append(tmp_df)

        clip_df['SAMPLING RATE'] = [44100] * clip_df.shape[0]
        clip_df['MANUAL ID'] = [row['MANUAL ID']] * clip_df.shape[0]
        clip_df.reset_index(inplace=True)
        df_split_save = df_split_save.append(clip_df, ignore_index=True)
        print(i, "out of", df.shape[0])

    print(df_split_save)


/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
!pip install pydub
from opensoundscape.audio import Audio
from pydub import AudioSegment
#AudioSegment.converter = "C:/Users/Siloux/Downloads/ffmpeg-master-latest-win64-gpl/ffmpeg-master-latest-win64-gpl/bin/ffmpeg.exe"
import os


clip_duration = 3
clip_overlap = 0
final_clip = None
clip_dir = './temp_clips/Cosmos_Data/Training_Xeno_Canto_2022'
# classes = labels
min_label_overlap = 0.1

df_split_save = pd.DataFrame(columns=['file', 'start_time', 'end_time', 'SAMPLING RATE', 'MANUAL ID', "CONFIDENCE"])

def convert_df_to_ops_df(row):
    offset = row["OFFSET"]
    duration = row["DURATION"]
    in_file_curr = row['IN FILE']
    file_name = in_file_curr.split('/')
    new_file = clip_dir + file_name[-1] + "_" + str(offset) + "s_" + str(offset+duration)+"s.wav"
    row['IN FILE'] = new_file
    
    if(not os.path.exists(new_file)):
        t1 = row["OFFSET"] * 1000
        # end time in milliseconds
        t2 = (row["OFFSET"] + 3) * 1000
        try:
            newAudio = AudioSegment.from_mp3(in_file_curr)
            newAudio = newAudio[t1:t2]
            new_file_object = newAudio.export(new_file, format="mp3")
            new_file_object.close()
        except Exception as e:
            print("failed on", in_file_curr)
            row['IN FILE'] = "TO DELETE"
        #newAudio.close()
    
    return row
    

df_refractor = df.apply(convert_df_to_ops_df, axis=1)
df_refractor = df_refractor[df_refractor["IN FILE"] != "TO DELETE"]
df_refractor

In [ ]:
df_split_save = df_refractor[["IN FILE", "OFFSET", "DURATION", "SAMPLE RATE", "MANUAL ID"]]
df_split_save["end_time"] = df_split_save["OFFSET"] + df_split_save["DURATION"]
df_split_save = df_split_save.rename(columns={"IN FILE": "file", "OFFSET": "start_time"})
df_split_save = df_split_save[['file', 'start_time', 'end_time', 'SAMPLE RATE', 'MANUAL ID']]

In [ ]:
df_split_save.to_csv(train_name)
df_split_save

In [ ]:
pd.set_option('display.width', 10000)
pd.set_option('display.expand_frame_repr', False)
df["merge file"] = df["IN FILE"].apply(lambda x: x.split("/")[-1])
df_split_save['CONFIDENCE'] = df_split_save['CONFIDENCE'].fillna(-1)
#for i in range(df.shape[0]):
##    if (i % 100 == 0):
#        print(i)
#    row = df.iloc[i]
#    rows_to_change = (df_split_save["file"].str.contains(row["merge file"])) & (df_split_save["start_time"] == (row["Start (s)"])) & (df_split_save["end_time"] == (row["End (s)"]))
#    df_split_save.loc[rows_to_change, "CONFIDENCE"]= row["Confidence"]
df_split_save

In [ ]:
df_split_save.drop_duplicates()

In [ ]:
import os
from os import listdir
from os.path import isfile, join
test_path = "./temp_clips/Cosmos_Data/Testing_Xeno_Canto_2022"
test_files = [f for f in listdir(test_path) if isfile(join(test_path, f))]
train_path = "./temp_clips/Cosmos_Data/Training_Xeno_Canto_2022"
train_files = [f for f in listdir(train_path) if isfile(join(train_path, f))]
overlapping_files = []
for item in train_files:
    if (item in test_files):
        path = train_path + "/" + item
        os.remove(path)
        overlapping_files.append(path)

df_split_save2 = df_split_save[~df_split_save["file"].isin(overlapping_files)]
df_split_save2.to_csv(train_name)       
print(len(overlapping_files))
df_split_save2

In [ ]:
test_path2 = "./temp_clips/Cosmos_Data/Testing_Xeno_Canto_2022"
test_files2 = [f for f in listdir(test_path2) if isfile(join(test_path2, f))]
train_path2 = "./temp_clips/Cosmos_Data/Training_Xeno_Canto_2022"
train_files2 = [f for f in listdir(train_path2) if isfile(join(train_path2, f))]
overlapping_files2 = []
for item in train_files2:
    if (item in test_files2):
        overlapping_files2.append(item)
overlapping_files2

In [ ]:
df_split_save2 = df_split_save[~df_split_save["file"].isin(overlapping_files)]
df_split_save2.to_csv(train_name)       
print(len(overlapping_files))

In [ ]:
#df_split_save2.loc[df_split_save2["CONFIDENCE"] == -1, "CONFIDENCE"] = 0
#df_split_save2 = df_split_save2.loc[df_split_save2["CONFIDENCE"] > 0.5]
df_split_save2.to_csv(train_name) 

In [ ]:
df_split_save2